In [1]:
import os, gc, torch

import numpy as np
import pandas as pd
# from models.load import TrainedModels

from utils.eval import save_iou_results
from utils.engine import xami_evaluate, get_iou_types
from models.load import get_trained_model
from utils.coco_eval import get_eval_params_dict
from data.datasets import  OurRadiologsitsDataset, collate_fn
from our_radiologist.load import get_anns
from utils.coco_utils import get_cocos, get_coco_api_from_dataset
from utils.eval import get_ap_ar
from utils.print import print_title
from utils.init import reproducibility, clean_memory_get_device
from data.load import get_datasets, get_dataloaders
from data.constants import XAMI_MIMIC_PATH, DEFAULT_REFLACX_LABEL_COLS
from utils.constants import full_iou_thrs, iou_thrs_5to95
from data.load  import seed_worker, get_dataloader_g
from tqdm import tqdm

## Suppress the assignement warning from pandas.
pd.options.mode.chained_assignment = None  # default='warn

## Supress user warning
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

%matplotlib inline

In [2]:
device = clean_memory_get_device()
reproducibility()

This notebook will running on device: [CUDA]


In [3]:
from enum import Enum

class TrainedModels(Enum):
    CXR_Clinial_fusion1_fusion2="val_ar_0_5436_ap_0_1911_test_ar_0_5476_ap_0_3168_epoch49_WithClincal_05-23-2022 12-06-22_CXR_Clinical_roi_heads_spatialisation"
    CXR_Clinical_fusion1 = "val_ar_0_5476_ap_0_1984_test_ar_0_6038_ap_0_2757_epoch41_WithClincal_05-30-2022 08-01-54_CXR_Clinical_fusion1"
    CXR_Clinical_fusion2= "val_ar_0_4369_ap_0_2098_test_ar_0_4940_ap_0_2218_epoch58_WithClincal_05-30-2022 13-58-43_CXR_Clinical_fusion2"
    CXR="val_ar_0_5659_ap_0_1741_test_ar_0_5390_ap_0_1961_epoch36_WithoutClincal_05-29-2022 12-29-51_CXR"

In [4]:
# ========================================For Training [pretrained_resnet50_without_clinical]========================================
# ModelSetup(use_clinical=False, use_custom_model=True, use_early_stop_model=True, name='pretrained_resnet50_without_clinical', best_ar_val_model_path=None, best_ap_val_model_path=None, final_model_path=None, backbone='resnet50', optimiser='sgd', lr=0.0005, weight_decay=5e-05, pretrained=True, record_training_performance=True, dataset_mode='unified')
# ===================================================================================================================================

# Best AP validation model has been saved to: [val_ar_0_5272_ap_0_2396_test_ar_0_6170_ap_0_1962_epoch44_WithoutClincal_04-19-2022 07-35-55_pretrained_resnet50_without_clinical]
# Best AR validation model has been saved to: [val_ar_0_5272_ap_0_2396_test_ar_0_6170_ap_0_1962_epoch44_WithoutClincal_04-19-2022 07-35-46_pretrained_resnet50_without_clinical]
# The final model has been saved to: [val_ar_0_3257_ap_0_1868_test_ar_0_4210_ap_0_1598_epoch100_WithoutClincal_04-19-2022 12-44-29_pretrained_resnet50_without_clinical]

# ===================================================================================================================================


# ========================================For Training [pretrained_resnet50_with_clinical]========================================
# ModelSetup(use_clinical=True, use_custom_model=True, use_early_stop_model=True, name='pretrained_resnet50_with_clinical', best_ar_val_model_path=None, best_ap_val_model_path=None, final_model_path=None, backbone='resnet50', optimiser='sgd', lr=0.0005, weight_decay=5e-05, pretrained=True, record_training_performance=True, dataset_mode='unified')
# ================================================================================================================================

# Best AP validation model has been saved to: [val_ar_0_4246_ap_0_2378_test_ar_0_4963_ap_0_1712_epoch98_WithClincal_04-19-2022 22-54-16_pretrained_resnet50_with_clinical]
# Best AR validation model has been saved to: [val_ar_0_5164_ap_0_1869_test_ar_0_5086_ap_0_1718_epoch64_WithClincal_04-19-2022 19-33-35_pretrained_resnet50_with_clinical]
# The final model has been saved to: [val_ar_0_3271_ap_0_1848_test_ar_0_3900_ap_0_1726_epoch100_WithClincal_04-19-2022 23-06-47_pretrained_resnet50_with_clinical]

# ================================================================================================================================

In [5]:
normal_iou_thrs = iou_thrs_5to95
all_range_iou_thrs = full_iou_thrs
score_thresholds = [0.05]

# Run evaluation.

In [6]:
for select_model in tqdm([
    TrainedModels.CXR_Clinical_fusion1,
    TrainedModels.CXR_Clinical_fusion2,
    TrainedModels.CXR
]):

    for score_thrs in score_thresholds:

        model, train_info, _, _ = get_trained_model(
            select_model,
            DEFAULT_REFLACX_LABEL_COLS,
            device,
            rpn_nms_thresh=0.3,
            box_detections_per_img=10,
            box_nms_thresh=0.2,
            rpn_score_thresh=0.0,
            box_score_thresh=score_thrs,
        )

        model.eval()

        iou_types = get_iou_types(model, train_info.model_setup)

        # dataset_params_dict = {
        #     "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
        #     "with_clinical": train_info.model_setup.use_clinical,
        #     "dataset_mode": train_info.model_setup.dataset_mode,
        #     "bbox_to_mask": True,
        #     "labels_cols": [
        #         "Enlarged cardiac silhouette",
        #         "Atelectasis",
        #         "Pleural abnormality",
        #         "Consolidation",
        #         "Pulmonary edema",
        #         #  'Groundglass opacity', # 6th disease.
        #     ],
        # } 

        model_setup = train_info.model_setup
        dataset_params_dict = {
            "XAMI_MIMIC_PATH": XAMI_MIMIC_PATH,
            "with_clinical": model_setup.use_clinical,
            "dataset_mode": model_setup.dataset_mode,
            "bbox_to_mask": model_setup.use_mask,
            "normalise_clinical_num": model_setup.normalise_clinical_num,
            "labels_cols": DEFAULT_REFLACX_LABEL_COLS,
        }

        detect_eval_dataset, train_dataset, val_dataset, test_dataset = get_datasets(
            dataset_params_dict=dataset_params_dict,
        )

        train_dataloader, val_dataloader, test_dataloader = get_dataloaders(
            train_dataset, val_dataset, test_dataset, batch_size=4,
        )

        train_coco, val_coco, test_coco = get_cocos(
            train_dataloader, val_dataloader, test_dataloader
        )

        radiologists_ann = get_anns("radiologists_annotated", detect_eval_dataset)

        radiologist_dataset = OurRadiologsitsDataset(
            detect_eval_dataset, radiologists_ann
        )
        radiologist_dataloader = torch.utils.data.DataLoader(
            radiologist_dataset,
            batch_size=4,
            shuffle=False,
            collate_fn=collate_fn,
            worker_init_fn=seed_worker,
            generator=get_dataloader_g(0),
        )

        radiologists_coco = get_coco_api_from_dataset(radiologist_dataloader.dataset)

        normal_eval_params_dict = get_eval_params_dict(
            detect_eval_dataset, iou_thrs=normal_iou_thrs,
        )

        all_range_eval_params_dict = get_eval_params_dict(
            detect_eval_dataset, iou_thrs=all_range_iou_thrs,
        )

        all_cat_ids = [None] + [
            detect_eval_dataset.disease_to_idx(d)
            for d in detect_eval_dataset.labels_cols
        ]

        for cat_id in all_cat_ids:
            cat_ids = (
                [
                    detect_eval_dataset.disease_to_idx(d)
                    for d in detect_eval_dataset.labels_cols
                ]
                if cat_id is None
                else [cat_id]
            )

            if not (cat_ids is None):
                all_range_eval_params_dict["bbox"].catIds = cat_ids
                all_range_eval_params_dict["segm"].catIds = cat_ids

            train_evaluator, _ = xami_evaluate(
                model,
                train_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=train_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            test_evaluator, _ = xami_evaluate(
                model,
                test_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=test_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            val_evaluator, _ = xami_evaluate(
                model,
                val_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=val_coco,
                iou_types=iou_types,
                # score_thres=score_thres,
            )

            radiologist_evaluator, _ = xami_evaluate(
                model,
                radiologist_dataloader,
                device=device,
                params_dict=all_range_eval_params_dict,
                coco=radiologists_coco,
                iou_types=iou_types,
            )

            if cat_id is None:
                disease_str = "all"
            else:
                disease_str = detect_eval_dataset.label_idx_to_disease(cat_id)

            save_iou_results(
                train_evaluator,
                f"train_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )
            save_iou_results(
                test_evaluator,
                f"test_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )
            save_iou_results(
                val_evaluator,
                f"val_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )
            save_iou_results(
                radiologist_evaluator,
                f"our_{disease_str}_score_thrs{score_thrs}",
                select_model.value,
            )

            train_ap_ar = get_ap_ar(
                train_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )

            test_ap_ar = get_ap_ar(
                test_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )
            val_ap_ar = get_ap_ar(
                val_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )
            our_ap_ar = get_ap_ar(
                radiologist_evaluator, areaRng="all", maxDets=10, iouThr=0.5,
            )

            df = pd.DataFrame(
                [
                    {
                        "dataset": "train",
                        f"AP@[IoBB = 0.50]": train_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": train_ap_ar["ar"],
                    },
                    {
                        "dataset": "test",
                        f"AP@[IoBB = 0.50]": test_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": test_ap_ar["ar"],
                    },
                    {
                        "dataset": "val",
                        f"AP@[IoBB = 0.50]": val_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": val_ap_ar["ar"],
                    },
                    {
                        "dataset": "our",
                        f"AP@[IoBB = 0.50]": our_ap_ar["ap"],
                        f"AR@[IoBB = 0.50]": our_ap_ar["ar"],
                    },
                ]
            )

            # train_ap_ar = get_ap_ar(
            #     train_evaluator, areaRng="all", maxDets=10, iouThr=[0.5, 0.95]
            # )

            # test_ap_ar = get_ap_ar(
            #     test_evaluator, areaRng="all", maxDets=10, iouThr=[0.5, 0.95]
            # )
            # val_ap_ar = get_ap_ar(
            #     val_evaluator, areaRng="all", maxDets=10, iouThr=[0.5, 0.95]
            # )
            # our_ap_ar = get_ap_ar(
            #     radiologist_evaluator, areaRng="all", maxDets=10, iouThr=[0.5, 0.95]
            # )

            # df = pd.DataFrame(
            #     [
            #         {
            #             "dataset": "train",
            #             f"AP@[IoBB = 0.50:0.95]": train_ap_ar["ap"],
            #             f"AR@[IoBB = 0.50:0.95]": train_ap_ar["ar"],
            #         },
            #         {
            #             "dataset": "test",
            #             f"AP@[IoBB = 0.50:0.95]": test_ap_ar["ap"],
            #             f"AR@[IoBB = 0.50:0.95]": test_ap_ar["ar"],
            #         },
            #         {
            #             "dataset": "val",
            #             f"AP@[IoBB = 0.50:0.95]": val_ap_ar["ap"],
            #             f"AR@[IoBB = 0.50:0.95]": val_ap_ar["ar"],
            #         },
            #         {
            #             "dataset": "our",
            #             f"AP@[IoBB = 0.50:0.95]": our_ap_ar["ap"],
            #             f"AR@[IoBB = 0.50:0.95]": our_ap_ar["ar"],
            #         },
            #     ]
            # )

            df.to_csv(
                os.path.join(
                    "eval_results",
                    f"{select_model.value}_{disease_str}_score_thrs{score_thrs}.csv",
                )
            )
            print_title(disease_str)
            print(df)


  0%|          | 0/3 [00:00<?, ?it/s]

Load custom model
Using pretrained backbone. mobilenet_v3
Using pretrained backbone. mobilenet_v3
Mask Hidden Layers 256
Using SGD as optimizer with lr=0.001
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
Evaluation:  [  0/117]  eta: 0:14:03  loss: 0.4940 (0.4940)  loss_classifier: 0.0221 (0.0221)  loss_box_reg: 0.0252 (0.0252)  loss_mask: 0.4264 (0.4264)  loss_objectness: 0.0173 (0.0173)  loss_rpn_box_reg: 0.0030 (0.0030)  model_time: 5.7970 (5.7970)  evaluator_time: 0.5899 (0.5899)  time: 7.2062  data: 0.6118  max mem: 1324
Evaluation:  [100/117]  eta: 0:00:22  loss: 0.4181 (0.4156)  loss_classifier: 0.0119 (0.0179)  loss_box_reg: 0.0093 (0.0138)  loss_mask: 0.3309 (0.3668)  loss_objectness: 0.0130 (0.0154)  loss_rpn_box_reg: 0.0012 (0.0017)  model_time: 0.1531 (0.2289)  evaluator_time: 0.3078 (0.3764)  time: 1.1810  data: 0.6200  max 

 33%|███▎      | 1/3 [27:41<55:23, 1661.54s/it]

  dataset  AP@[IoBB = 0.50]  AR@[IoBB = 0.50]
0   train          0.248362          0.623656
1    test          0.222601          0.722222
2     val          0.110749          0.578947
3     our          0.000000          0.000000
Load custom model
Using pretrained backbone. mobilenet_v3
Mask Hidden Layers 256
Using SGD as optimizer with lr=0.001
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
Evaluation:  [  0/117]  eta: 0:02:39  loss: 0.3457 (0.3457)  loss_classifier: 0.0117 (0.0117)  loss_box_reg: 0.0161 (0.0161)  loss_mask: 0.3003 (0.3003)  loss_objectness: 0.0149 (0.0149)  loss_rpn_box_reg: 0.0026 (0.0026)  model_time: 0.1456 (0.1456)  evaluator_time: 0.3167 (0.3167)  time: 1.3643  data: 0.6212  max mem: 1836
Evaluation:  [100/117]  eta: 0:00:18  loss: 0.3075 (0.3654)  loss_classifier: 0.0097 (0.0117)  loss_box_reg: 0.0097 (0.0125)  l

 67%|██████▋   | 2/3 [50:46<24:58, 1498.84s/it]

  dataset  AP@[IoBB = 0.50]  AR@[IoBB = 0.50]
0   train          0.200538          0.516129
1    test          0.154344          0.555556
2     val          0.090480          0.368421
3     our          0.000000          0.000000
Load custom model
Using pretrained backbone. mobilenet_v3
Mask Hidden Layers 256
Using SGD as optimizer with lr=0.001
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
creating index...
index created!
Evaluation:  [  0/117]  eta: 0:03:08  loss: 0.5335 (0.5335)  loss_classifier: 0.0379 (0.0379)  loss_box_reg: 0.0263 (0.0263)  loss_mask: 0.4443 (0.4443)  loss_objectness: 0.0212 (0.0212)  loss_rpn_box_reg: 0.0038 (0.0038)  model_time: 0.2112 (0.2112)  evaluator_time: 0.5587 (0.5587)  time: 1.6140  data: 0.7735  max mem: 1836
Evaluation:  [100/117]  eta: 0:00:23  loss: 0.5198 (0.4434)  loss_classifier: 0.0196 (0.0275)  loss_box_reg: 0.0126 (0.0187)  l

100%|██████████| 3/3 [1:18:31<00:00, 1570.52s/it]

  dataset  AP@[IoBB = 0.50]  AR@[IoBB = 0.50]
0   train          0.181063          0.655914
1    test          0.092211          0.666667
2     val          0.132017          0.578947
3     our          0.000000          0.000000


In [7]:
select_model

<TrainedModels.CXR: 'val_ar_0_5659_ap_0_1741_test_ar_0_5390_ap_0_1961_epoch36_WithoutClincal_05-29-2022 12-29-51_CXR'>